In [5]:
from process_config import *
from rime_functions import *
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from dask.distributed import Client
import glob
import numpy as np
import pandas as pd
import pyam
import re
import time
import xarray as xr
import dask

# Overview

**Inputs:**
 - Emissions or GMT scenarios from IAM.  
 - Climate impacts database aggregated to region (e.g. country or R10) by Global Warming Level.  

**Outputs:**
 - Table data of climate impacts through time in IAMC format, per scenario.

Steps  
1. Import climate impacts database. NetCDF 4D file with different impacts as variables.
2. Import the scenarios to be run. Can be global mean temp scenarios or Emissions|CO2 scenarios.
3. Preprocessing steps.
4. Run.


### Import the climate impacts database files


In [12]:
filesall = glob.glob(fname_input_climate)
files = filesall[12:14]
f = files[0]
# load input climate impacts data file
ds = xr.open_mfdataset(f)
ds = ds.sel(year=years)
ds

<xarray.Dataset>
Dimensions:                                     (gmt: 251, year: 9, ssp: 3,
                                                 region: 226)
Coordinates:
  * gmt                                         (gmt) float64 1.2 1.206 ... 3.5
  * year                                        (year) int64 2015 2025 ... 2095
  * ssp                                         (ssp) object 'SSP1' ... 'SSP3'
  * region                                      (region) object 'AFG' ... 'wo...
Data variables:
    sdii|Exposure|Land area                     (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>
    sdii|Exposure|Population                    (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>
    sdii|Exposure|Population|%                  (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>
    sdii|Hazard|Absolute|Land area weighted     (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>
    sdii|Hazard|Absolute|Population weighted    (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>
    sdii|Hazard|Risk score|Population weighted  (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>

In [14]:
varis = list(ds.data_vars.keys())[:lvaris]
dsi = ds[varis]
print(f"# of variables = {len(varis)}")


# of variables = 6


### Import scenarios data

Load an IAMC scenarios dataset.  
Decide whether using global mean temperature or CO2 mode.  
Assign SSPs if missing and fix duplicate temperatures.  


In [13]:
df_scens_in = pyam.IamDataFrame(fname_input_scenarios)
# SCENARIO DATA PRE-PROCESSING
# Filter for temperature variable

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file C:\Users\byers\IIASA\ECE.prog - Documents\Research Theme - NEXUS\Hotspots_Explorer_2p0\rcre_testing\testing_2\emissions_temp_AR6_small.xlsx
pyam.core - INFO: Reading meta indicators


In [15]:
df_scens_in

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : AIM/CGE 2.0, AIM/CGE 2.1, AIM/CGE 2.2, ... WITCH-GLOBIOM 4.4 (44)
 * scenario : 1.5C, 2.5C, 2C, 2CNow_Gradual, ... WB2C (559)
Timeseries data coordinates:
   region   : World (1)
   variable : AR6 climate diagnostics|Infilled|Emissions|CO2, ... (3)
   unit     : K, Mt CO2/yr (2)
   year     : 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, ... 2100 (86)
Meta indicators:
   Category (object) C3, C5, C6, C7, C4, C1, C2, C8 (8)
   Category_name (object) C3: limit warming to 2°C (>67%), ... (8)
   Category_subset (object) C3y_+veGHGs, C5, C6, C7, C4, C1a_NZGHGs, ... C8 (10)
   Subset_Ch4 (object) Limit to 2C (>67%) immediate 2020 action, ... (6)
   Category_Vetting_historical (object) C3, C5, C6, C7, C4, C1, C2, C8 (8)
   ...

In [55]:
mode = 'GMT'
if mode == "GMT":
    dfp = df_scens_in.filter(variable=temp_variable)
elif mode == "CO2":
    dfp = prepare_cumulative(df_scens_in, years=years, use_dask=True)
    ts = dfp.timeseries().apply(co2togmt_simple)
    ts = pyam.IamDataFrame(ts)
    ts.rename(
        {
            "variable": {ts.variable[0]: "RIME|GSAT_tcre"},
            "unit": {ts.unit[0]: "°C"},
        },
        inplace=True,
    )
    # Export data to check error and relationships
    # ts.append(dfp).to_csv('c://users//byers//downloads//tcre_gmt_output.csv')
    dfp = ts
    dfp.meta = df_scens_in.meta.copy()
dfp = dfp.filter(year=years)

In [56]:
few_scenarios = False
very_few_scenarios = False

if few_scenarios:
    dfp = dfp.filter(Category=["C1*"])
    if very_few_scenarios:
        dfp = dfp.filter(model="REMIND 2.1*", scenario="*") 
        
        
#  Assign SSP to meta and select SSP2 in case SSP not present in name
dfp = ssp_helper(dfp, ssp_meta_col="Ssp_family", default_ssp="SSP2")

# Fix duplicate temperatures
dft = dfp.timeseries().reset_index()
dft = dft.apply(fix_duplicate_temps, years=years, axis=1)
dft.reset_index(inplace=True, drop=True)
dft

,model,scenario,region,variable,unit,ssp_family,2015,2025,2035,2045,2055,2065,2075,2085,2095
0,AIM/CGE 2.0,SSP1-26,World,AR6 climate diagnostics|Surface Temperature (G...,K,SSP1,1.116,1.379,1.569,1.645,1.679,1.677,1.663,1.648,1.632
1,AIM/CGE 2.0,SSP1-34,World,AR6 climate diagnostics|Surface Temperature (G...,K,SSP1,1.116,1.377,1.612,1.787,1.909,1.987,2.031,2.066,2.089
2,AIM/CGE 2.0,SSP1-45,World,AR6 climate diagnostics|Surface Temperature (G...,K,SSP1,1.116,1.377,1.654,1.950,2.214,2.393,2.521,2.598,2.653
3,AIM/CGE 2.0,SSP1-Baseline,World,AR6 climate diagnostics|Surface Temperature (G...,K,SSP1,1.116,1.386,1.696,2.013,2.332,2.605,2.857,3.088,3.313
4,AIM/CGE 2.0,SSP4-26,World,AR6 climate diagnostics|Surface Temperature (G...,K,SSP4,1.115,1.349,1.523,1.585,1.607,1.614,1.595,1.572,1.547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,WITCH-GLOBIOM 4.4,CD-LINKS_NPi,World,AR6 climate diagnostics|Surface Temperature (G...,K,SSP2,1.115,1.345,1.605,1.943,2.305,2.644,3.008,3.362,3.727
1198,WITCH-GLOBIOM 4.4,CD-LINKS_NPi2020_1000,World,AR6 climate diagnostics|Surface Temperature (G...,K,SSP2,1.115,1.360,1.417,1.426,1.441,1.442,1.419,1.398,1.367
1199,WITCH-GLOBIOM 4.4,CD-LINKS_NPi2020_1600,World,AR6 climate diagnostics|Surface Temperature (G...,K,SSP2,1.115,1.350,1.461,1.523,1.569,1.614,1.647,1.658,1.656
1200,WITCH-GLOBIOM 4.4,CD-LINKS_NPi2020_400,World,AR6 climate diagnostics|Surface Temperature (G...,K,SSP2,1.115,1.365,1.395,1.362,1.315,1.268,1.205,1.132,1.057


### START PROCESSING

In [51]:
start = time.time()
year_res = 10
parallel = True
if parallel:
    """
    For parallel processing, convert dft as a wide IAMC pd.Dataframe
    into a dask.DataFrame.
    """
    ddf = dd.from_pandas(dft, npartitions=1000)

    # dfx = dft.iloc[0].squeeze()  # FOR DEBUIGGING THE FUNCTION
    outd = ddf.apply(
        table_impacts_gmt, dsi=dsi, axis=1, meta=("result", None)
    )

    with ProgressBar():
        # try:
        df_new = outd.compute(num_workers=num_workers)
else:
    df_new = dft.apply(table_impacts_gmt, dsi=dsi, axis=1)

expandeddGMT = pd.concat([df_new[x] for x in df_new.index])
print(f" Done:  {time.time()-start}")

filename = f"RIME_output_{region}_{year_res}yr.csv"

# expandedd.to_csv(filename, encoding="utf-8", index=False)
print(f" Saved: {region} yrs={year_res}\n  {time.time()-start}")
print(f"{len(dsi.data_vars)} variables, {len(dfp.meta)} scenarios")

[########################################] | 100% Completed | 3.31 sms
 Done:  3.3233566284179688
 Saved: COUNTRIES yrs=10
  3.3233566284179688
6 variables, 8 scenarios


In [48]:
expandedd

,model,scenario,region,variable,unit,2015,2025,2035,2045,2055,2065,2075,2085,2095
0,REMIND 2.1,CEMICS_GDPgrowth_1p5,AFG,RIME|sdii|Exposure|Land area,km2,0.000000,63.643730,763.724757,1081.943405,1145.587135,1018.299676,700.081027,318.218649,0.000000
1,REMIND 2.1,CEMICS_GDPgrowth_1p5,AGO,RIME|sdii|Exposure|Land area,km2,0.000000,104.598105,1255.177258,1778.167781,1882.765886,1673.569677,1150.579153,522.990524,0.000000
2,REMIND 2.1,CEMICS_GDPgrowth_1p5,ALB,RIME|sdii|Exposure|Land area,km2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,REMIND 2.1,CEMICS_GDPgrowth_1p5,AND,RIME|sdii|Exposure|Land area,km2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,REMIND 2.1,CEMICS_GDPgrowth_1p5,ARE,RIME|sdii|Exposure|Land area,km2,11916.028165,1499.582695,2122.578146,2405.757896,2462.393846,2349.121946,2065.942196,1726.126495,2141.152173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,REMIND 2.1,R2p1_SSP5-PkBudg900,YEM,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.176877,0.247782,0.232719,0.226411,0.226991,0.229761,0.233568,0.238392,0.240822
1352,REMIND 2.1,R2p1_SSP5-PkBudg900,ZAF,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.262779,0.235003,0.229794,0.228257,0.229939,0.232715,0.236093,0.240023,0.244809
1353,REMIND 2.1,R2p1_SSP5-PkBudg900,ZMB,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.012162,0.011360,0.011437,0.011514,0.011561,0.011572,0.011556,0.011512,0.011485
1354,REMIND 2.1,R2p1_SSP5-PkBudg900,ZWE,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.009713,0.005452,0.006224,0.006454,0.006260,0.005943,0.005576,0.005164,0.004891


In [52]:
expandeddGMT

,model,scenario,region,variable,unit,2015,2025,2035,2045,2055,2065,2075,2085,2095
0,REMIND 2.1,CEMICS_GDPgrowth_1p5,AFG,RIME|sdii|Exposure|Land area,km2,0.000000,0.000000,318.218649,445.506108,445.506108,127.287459,0.000000,0.000000,0.000000
1,REMIND 2.1,CEMICS_GDPgrowth_1p5,AGO,RIME|sdii|Exposure|Land area,km2,0.000000,0.000000,522.990524,732.186734,732.186734,209.196210,0.000000,0.000000,0.000000
2,REMIND 2.1,CEMICS_GDPgrowth_1p5,ALB,RIME|sdii|Exposure|Land area,km2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,REMIND 2.1,CEMICS_GDPgrowth_1p5,AND,RIME|sdii|Exposure|Land area,km2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,REMIND 2.1,CEMICS_GDPgrowth_1p5,ARE,RIME|sdii|Exposure|Land area,km2,18898.082445,7726.795597,1726.126495,1839.398395,1839.398395,1556.218645,3537.563029,7726.795597,11916.028165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,REMIND 2.1,R2p1_SSP5-PkBudg900,YEM,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.127871,0.205334,0.244327,0.240007,0.243701,0.223753,0.196962,0.167887,0.138906
1352,REMIND 2.1,R2p1_SSP5-PkBudg900,ZAF,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.281809,0.252662,0.235246,0.234885,0.238325,0.249184,0.260808,0.273294,0.285621
1353,REMIND 2.1,R2p1_SSP5-PkBudg900,ZMB,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.012770,0.011914,0.011435,0.011490,0.011505,0.011784,0.012127,0.012493,0.012855
1354,REMIND 2.1,R2p1_SSP5-PkBudg900,ZWE,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.012558,0.007898,0.005513,0.005625,0.005253,0.006182,0.007450,0.008794,0.010106
